**ICP - 10**

**1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”) **

In [3]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('/content/Sentiment (3).csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

291/291 - 36s - loss: 0.8185 - accuracy: 0.6467 - 36s/epoch - 123ms/step
144/144 - 2s - loss: 0.7502 - accuracy: 0.6723 - 2s/epoch - 11ms/step
0.7501501441001892
0.6723459959030151
['loss', 'accuracy']


In [4]:
model.save("sentiment_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
import tweepy
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

# Load the saved model
model = load_model("/content/sentiment_model.h5")

# Define a function for preprocessing text
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-z0-9\s]', '', text)
    return text

# Example new text data
new_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing. @realDonaldTrump"

# Preprocess the new text data
new_text = preprocess_text(new_text)

# Tokenize and pad the new text data
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts([new_text])
X_new = tokenizer.texts_to_sequences([new_text])
X_new = pad_sequences(X_new, maxlen=model.input_shape[1])

# Make predictions
predictions = model.predict(X_new)

# Determine the sentiment based on the prediction
sentiments = ['Negative', 'Neutral', 'Positive']
predicted_sentiment = sentiments[predictions.argmax()]

# Print the result
print("Predicted Sentiment: " + predicted_sentiment)


1/1 [==============================] - 0s 270ms/step
Predicted Sentiment: Negative


**2. Apply GridSearchCV on the source code provided in the class**

In [6]:
!pip install scikeras

In [7]:
from scikeras.wrappers import KerasClassifier

In [8]:
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier

# Assuming the data loading and preprocessing steps are the same

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
# Assuming tokenizer fitting and text preprocessing is done here

def createmodel(optimizer='adam'):
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define the KerasClassifier with the build_fn as our model creation function
model = KerasClassifier(model=createmodel, verbose=2)

# Define hyperparameters to tune
param_grid = {
    'batch_size': [32, 64],
    'epochs': [1, 2],
    'optimizer': ['adam', 'rmsprop']
}

# Initialize GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
# Fit GridSearchCV
grid_result = grid.fit(X_train, Y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


194/194 - 34s - loss: 0.8732 - accuracy: 0.6310 - 34s/epoch - 175ms/step
97/97 - 1s - 1s/epoch - 10ms/step


194/194 - 28s - loss: 0.8607 - accuracy: 0.6281 - 28s/epoch - 144ms/step
97/97 - 1s - 979ms/epoch - 10ms/step


194/194 - 25s - loss: 0.8544 - accuracy: 0.6285 - 25s/epoch - 128ms/step
97/97 - 1s - 1s/epoch - 10ms/step


194/194 - 25s - loss: 0.8573 - accuracy: 0.6308 - 25s/epoch - 126ms/step
97/97 - 1s - 999ms/epoch - 10ms/step


194/194 - 25s - loss: 0.8629 - accuracy: 0.6313 - 25s/epoch - 130ms/step
97/97 - 1s - 1s/epoch - 10ms/step


...

Epoch 1/2
97/97 - 17s - loss: 0.8922 - accuracy: 0.6131 - 17s/epoch - 170ms/step
Epoch 2/2
97/97 - 10s - loss: 0.7124 - accuracy: 0.6956 - 10s/epoch - 105ms/step
49/49 - 1s - 690ms/epoch - 14ms/step


Epoch 1/2
97/97 - 18s - loss: 0.8873 - accuracy: 0.6197 - 18s/epoch - 181ms/step
Epoch 2/2
97/97 - 10s - loss: 0.7319 - accuracy: 0.6836 - 10s/epoch - 98ms/step
49/49 - 1s - 631ms/epoch - 13ms/step


Epoch 1/2
97/97 - 17s - loss: 0.8900 - accuracy: 0.6194 - 17s/epoch - 171ms/step
Epoch 2/2
97/97 - 9s - loss: 0.7335 - accuracy: 0.6854 - 9s/epoch - 93ms/step
49/49 - 1s - 673ms/epoch - 14ms/step


Epoch 1/2
97/97 - 16s - loss: 0.8853 - accuracy: 0.6183 - 16s/epoch - 163ms/step
Epoch 2/2
97/97 - 10s - loss: 0.7133 - accuracy: 0.6904 - 10s/epoch - 102ms/step
49/49 - 1s - 626ms/epoch - 13ms/step


Epoch 1/2
146/146 - 22s - loss: 0.8484 - accuracy: 0.6341 - 22s/epoch - 149ms/step
Epoch 2/2
146/146 - 15s - loss: 0.6949 - accuracy: 0.7050 - 15s/epoch - 101ms/step
Best: 0.676315 using {'batch_size': 64, 'epochs': 2, 'optimizer': 'rmsprop'}
